In [1]:
import glob
import os.path as osp
import uproot
import awkward as ak
import torch
import numpy as np
import random
import tqdm
from torch_geometric.data import Data, Dataset

import numpy as np
import subprocess
import tqdm
from tqdm import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random
from torch_geometric.nn import knn_graph
import torch.nn.functional as F

In [2]:
import numpy as np
import subprocess
import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn


from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random

def find_highest_branch(path, base_name):
    with uproot.open(path) as f:
        # Find keys that exactly match the base_name (not containing other variations)
        branches = [k for k in f.keys() if k.startswith(base_name + ';')]
        
        # Sort and select the highest-numbered branch
        sorted_branches = sorted(branches, key=lambda x: int(x.split(';')[-1]))
        return sorted_branches[-1] if sorted_branches else None
class CCV1(Dataset):
    r'''
        input: layer clusters

    '''

    url = '/dummy/'

    def __init__(self, root, transform=None, max_events=1e8, inp = 'train'):
        super(CCV1, self).__init__(root, transform)
        self.step_size = 500
        self.inp = inp
        self.max_events = max_events
        self.fill_data(max_events)

    def fill_data(self,max_events):
        counter = 0
        arrLens0 = []
        arrLens1 = []

        print("### Loading data")
        for fi,path in enumerate(tqdm.tqdm(self.raw_paths)):


            if self.inp == 'train':
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            elif self.inp == 'val':
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            else:
                cluster_path = find_highest_branch(path, 'clusters')
                sim_path = find_highest_branch(path, 'simtrackstersCP')
            
            crosstree =  uproot.open(path)[cluster_path]
            crosscounter = 0
            for array in uproot.iterate(f"{path}:{sim_path}", ["vertices_x", "vertices_y", "vertices_z", 
            "vertices_energy", "vertices_multiplicity", "vertices_time", "vertices_indexes", "barycenter_x", "barycenter_y", "barycenter_z"], step_size=self.step_size):
            
                tmp_stsCP_vertices_x = array['vertices_x']
                tmp_stsCP_vertices_y = array['vertices_y']
                tmp_stsCP_vertices_z = array['vertices_z']
                tmp_stsCP_vertices_energy = array['vertices_energy']
                tmp_stsCP_vertices_time = array['vertices_time']
                tmp_stsCP_vertices_indexes = array['vertices_indexes']
                tmp_stsCP_barycenter_x = array['barycenter_x']
                tmp_stsCP_barycenter_y = array['barycenter_y']
                tmp_stsCP_barycenter_z = array['barycenter_z']


                tmp_stsCP_vertices_multiplicity = array['vertices_multiplicity']
                
                # weighted energies (A LC appears in its caloparticle assignment array as the energy it contributes not full energy)
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy * tmp_stsCP_vertices_multiplicity
                
                self.step_size = min(self.step_size,len(tmp_stsCP_vertices_x))


                # Code block for reading from other tree
                tmp_all_vertices_layer_id = crosstree['cluster_layer_id'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                #tmp_all_vertices_radius = crosstree['cluster_radius'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_noh = crosstree['cluster_number_of_hits'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_eta = crosstree['position_eta'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                tmp_all_vertices_phi = crosstree['position_phi'].array(entry_start=crosscounter*self.step_size,entry_stop=(crosscounter+1)*self.step_size)
                crosscounter += 1

                layer_id_list = []
                radius_list = []
                noh_list = []
                eta_list = []
                phi_list = []
                for evt_row in range(len(tmp_all_vertices_noh)):
                    #print("Event no: %i"%evt_row)
                    #print("There are %i particles in this event"%len(tmp_stsCP_vertices_indexes[evt_row]))
                    layer_id_list_one_event = []
                    #radius_list_one_event = []
                    noh_list_one_event = []
                    eta_list_one_event = []
                    phi_list_one_event = []
                    for particle in range(len(tmp_stsCP_vertices_indexes[evt_row])):
                        #print("Particle no: %i"%particle)
                        #print("A")
                        #print(np.array(tmp_all_vertices_radius[evt_row]).shape)
                        #print("B")
                        #print(np.array(tmp_stsCP_vertices_indexes[evt_row][particle]).shape)
                        #print("C")
                        tmp_stsCP_vertices_layer_id_one_particle = tmp_all_vertices_layer_id[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #tmp_stsCP_vertices_radius_one_particle = tmp_all_vertices_radius[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_noh_one_particle = tmp_all_vertices_noh[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_eta_one_particle = tmp_all_vertices_eta[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        tmp_stsCP_vertices_phi_one_particle = tmp_all_vertices_phi[evt_row][tmp_stsCP_vertices_indexes[evt_row][particle]]
                        #print(tmp_stsCP_vertices_radius_one_particle)
                        layer_id_list_one_event.append(tmp_stsCP_vertices_layer_id_one_particle)
                        #radius_list_one_event.append(tmp_stsCP_vertices_radius_one_particle)
                        noh_list_one_event.append(tmp_stsCP_vertices_noh_one_particle)
                        eta_list_one_event.append(tmp_stsCP_vertices_eta_one_particle)
                        phi_list_one_event.append(tmp_stsCP_vertices_phi_one_particle)
                    layer_id_list.append(layer_id_list_one_event)
                    #radius_list.append(radius_list_one_event)
                    noh_list.append(noh_list_one_event)
                    eta_list.append(eta_list_one_event)
                    phi_list.append(phi_list_one_event)
                tmp_stsCP_vertices_layer_id = ak.Array(layer_id_list)                
                #tmp_stsCP_vertices_radius = ak.Array(radius_list)                
                tmp_stsCP_vertices_noh = ak.Array(noh_list)                
                tmp_stsCP_vertices_eta = ak.Array(eta_list)                
                tmp_stsCP_vertices_phi = ak.Array(phi_list)                

                


                

                
                #SHOULD BE LEN(E) >= 2 for MULTI particles
                skim_mask = []
                for e in tmp_stsCP_vertices_x:
                    if 2 <= len(e) <= 5: #<------ only train on samples with > 1 particle
                        skim_mask.append(True)
                    else:
                        skim_mask.append(False)
                tmp_stsCP_vertices_x = tmp_stsCP_vertices_x[skim_mask]
                tmp_stsCP_vertices_y = tmp_stsCP_vertices_y[skim_mask]
                tmp_stsCP_vertices_z = tmp_stsCP_vertices_z[skim_mask]
                tmp_stsCP_vertices_energy = tmp_stsCP_vertices_energy[skim_mask]
                tmp_stsCP_vertices_time = tmp_stsCP_vertices_time[skim_mask]
                tmp_stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id[skim_mask]
                #tmp_stsCP_vertices_radius = tmp_stsCP_vertices_radius[skim_mask]
                tmp_stsCP_vertices_noh = tmp_stsCP_vertices_noh[skim_mask]
                tmp_stsCP_vertices_eta = tmp_stsCP_vertices_eta[skim_mask]
                tmp_stsCP_vertices_phi = tmp_stsCP_vertices_phi[skim_mask]
                tmp_stsCP_vertices_indexes = tmp_stsCP_vertices_indexes[skim_mask]
                tmp_stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity[skim_mask]

                if counter == 0:
                    self.stsCP_vertices_x = tmp_stsCP_vertices_x
                    self.stsCP_vertices_y = tmp_stsCP_vertices_y
                    self.stsCP_vertices_z = tmp_stsCP_vertices_z
                    self.stsCP_vertices_energy = tmp_stsCP_vertices_energy
                    self.stsCP_vertices_time = tmp_stsCP_vertices_time
                    self.stsCP_vertices_layer_id = tmp_stsCP_vertices_layer_id
                    #self.stsCP_vertices_radius = tmp_stsCP_vertices_radius
                    self.stsCP_vertices_noh = tmp_stsCP_vertices_noh
                    self.stsCP_vertices_eta = tmp_stsCP_vertices_eta
                    self.stsCP_vertices_phi = tmp_stsCP_vertices_phi
                    self.stsCP_vertices_indexes = tmp_stsCP_vertices_indexes
                    self.stsCP_barycenter_x = tmp_stsCP_barycenter_x
                    self.stsCP_barycenter_y = tmp_stsCP_barycenter_y
                    self.stsCP_barycenter_z = tmp_stsCP_barycenter_z
                    self.stsCP_vertices_multiplicity = tmp_stsCP_vertices_multiplicity
                else:
                    self.stsCP_vertices_x = ak.concatenate((self.stsCP_vertices_x,tmp_stsCP_vertices_x))
                    self.stsCP_vertices_y = ak.concatenate((self.stsCP_vertices_y,tmp_stsCP_vertices_y))
                    self.stsCP_vertices_z = ak.concatenate((self.stsCP_vertices_z,tmp_stsCP_vertices_z))
                    self.stsCP_vertices_energy = ak.concatenate((self.stsCP_vertices_energy,tmp_stsCP_vertices_energy))
                    self.stsCP_vertices_time = ak.concatenate((self.stsCP_vertices_time,tmp_stsCP_vertices_time))
                    self.stsCP_vertices_layer_id = ak.concatenate((self.stsCP_vertices_layer_id,tmp_stsCP_vertices_layer_id))
                    #self.stsCP_vertices_radius = ak.concatenate((self.stsCP_vertices_radius,tmp_stsCP_vertices_radius))
                    self.stsCP_vertices_noh = ak.concatenate((self.stsCP_vertices_noh,tmp_stsCP_vertices_noh))
                    self.stsCP_vertices_eta = ak.concatenate((self.stsCP_vertices_eta,tmp_stsCP_vertices_eta))
                    self.stsCP_vertices_phi = ak.concatenate((self.stsCP_vertices_phi,tmp_stsCP_vertices_phi))
                    self.stsCP_vertices_indexes = ak.concatenate((self.stsCP_vertices_indexes,tmp_stsCP_vertices_indexes))
                    self.stsCP_barycenter_x = ak.concatenate((self.stsCP_barycenter_x,tmp_stsCP_barycenter_x))
                    self.stsCP_barycenter_y = ak.concatenate((self.stsCP_barycenter_y,tmp_stsCP_barycenter_y))
                    self.stsCP_barycenter_z = ak.concatenate((self.stsCP_barycenter_z,tmp_stsCP_barycenter_z))
                    self.stsCP_vertices_multiplicity = ak.concatenate((self.stsCP_vertices_multiplicity, tmp_stsCP_vertices_multiplicity))

                #print(len(self.stsCP_vertices_x))
                counter += 1
                if len(self.stsCP_vertices_x) > max_events:
                    print(f"Reached {max_events}!")
                    break
            if len(self.stsCP_vertices_x) > max_events:
                break
     
            
            
    def download(self):
        raise RuntimeError(
            'Dataset not found. Please download it from {} and move all '
            '*.z files to {}'.format(self.url, self.raw_dir))

    def len(self):
        return len(self.stsCP_vertices_x)

    @property
    def raw_file_names(self):
        raw_files = sorted(glob.glob(osp.join(self.raw_dir, '*.root')))
        
        #raw_files = [osp.join(self.raw_dir, 'step3_NTUPLE.root')]

        return raw_files

    @property
    def processed_file_names(self):
        return []


    def get(self, idx):
        def reconstruct_array(grouped_indices):
            # Find the maximum index to determine the array length
            max_index = max(max(indices) for indices in grouped_indices.values())

            # Initialize an array with the correct size, filled with a placeholder (e.g., -1)
            reconstructed = [-1] * (max_index + 1)

            # Populate the array based on the dictionary
            for value, indices in grouped_indices.items():
                for idx in indices:
                    reconstructed[idx] = value

            return reconstructed
        edge_index = torch.empty((2,0), dtype=torch.long)
 
        lc_x = self.stsCP_vertices_x[idx]
        #print(ak.to_numpy(lc_x[0]).shape)
        #print(ak.to_numpy(lc_x[1]).shape)
        flat_lc_x = np.expand_dims(np.array(ak.flatten(lc_x)),axis=1)
        lc_y = self.stsCP_vertices_y[idx]
        flat_lc_y = np.expand_dims(np.array(ak.flatten(lc_y)),axis=1)
        lc_z = self.stsCP_vertices_z[idx]
        flat_lc_z = np.expand_dims(np.array(ak.flatten(lc_z)),axis=1)
        lc_e = self.stsCP_vertices_energy[idx]
        flat_lc_e = np.expand_dims(np.array(ak.flatten(lc_e)),axis=1)     
        lc_t = self.stsCP_vertices_time[idx]
        flat_lc_t = np.expand_dims(np.array(ak.flatten(lc_t)),axis=1)  
        lc_layer_id = self.stsCP_vertices_layer_id[idx]
        flat_lc_layer_id = np.expand_dims(np.array(ak.flatten(lc_layer_id)),axis=1)  
        #lc_radius = self.stsCP_vertices_radius[idx]
        #flat_lc_radius = np.expand_dims(np.array(ak.flatten(lc_radius)),axis=1)  
        lc_noh = self.stsCP_vertices_noh[idx]
        flat_lc_noh = np.expand_dims(np.array(ak.flatten(lc_noh)),axis=1)  
        lc_eta = self.stsCP_vertices_eta[idx]
        flat_lc_eta = np.expand_dims(np.array(ak.flatten(lc_eta)),axis=1)  
        lc_phi = self.stsCP_vertices_phi[idx]
        flat_lc_phi = np.expand_dims(np.array(ak.flatten(lc_phi)),axis=1)  
        
        lc_indexes = self.stsCP_vertices_indexes[idx]
        flat_lc_indexes = np.expand_dims(np.array(ak.flatten(lc_indexes)),axis=1)  
        lc_multiplicity = self.stsCP_vertices_multiplicity[idx]
        flat_lc_multiplicity = np.expand_dims(np.array(ak.flatten(lc_multiplicity)),axis=1) 

        
        mask = np.zeros_like(flat_lc_indexes, dtype=bool)

        # Loop over each unique index.
        for unique_idx in np.unique(flat_lc_indexes):
            # Find the positions where this index occurs.
            positions = np.where(flat_lc_indexes == unique_idx)[0]
            # If there is only one occurrence, keep it.
            if positions.size == 1:
                mask[positions[0]] = True
            else:
                # If repeated, select the position with the minimum multiplicity.
                pos_to_keep = positions[np.argmin(flat_lc_multiplicity[positions])]
                mask[pos_to_keep] = True

        # Now use this mask to filter all arrays:
        flat_lc_x = flat_lc_x[mask].reshape(-1, 1)
        flat_lc_y = flat_lc_y[mask].reshape(-1, 1)
        flat_lc_z = flat_lc_z[mask].reshape(-1, 1)
        flat_lc_e = flat_lc_e[mask].reshape(-1, 1)
        flat_lc_t = flat_lc_t[mask].reshape(-1, 1)
        flat_lc_layer_id = flat_lc_layer_id[mask].reshape(-1, 1)
        flat_lc_noh = flat_lc_noh[mask].reshape(-1, 1)
        flat_lc_eta = flat_lc_eta[mask].reshape(-1, 1)
        flat_lc_phi = flat_lc_phi[mask].reshape(-1, 1)
        flat_lc_indexes = flat_lc_indexes[mask].reshape(-1, 1)
        flat_lc_multiplicity = flat_lc_multiplicity[mask].reshape(-1, 1)
                                                               

        flat_lc_feats = np.concatenate((flat_lc_x,flat_lc_y,flat_lc_z,flat_lc_e,\
                                        flat_lc_layer_id,flat_lc_noh,flat_lc_eta,flat_lc_phi),axis=-1)    

        
        # Determine the maximum vertex index over all sub-arrays
        max_index = max(np.max(arr) for arr in lc_indexes)

        # Initialize the association array and a tracker for the best multiplicity.
        assoc_array = -1 * np.ones(max_index + 1, dtype=int)
        best_mult = np.full(max_index + 1, np.inf)

        # Loop over each sub-array and its multiplicity values.
        for sub_idx, (indexes, mult_values) in enumerate(zip(lc_indexes, lc_multiplicity)):
            for pos, vertex_index in enumerate(indexes):
                candidate_mult = mult_values[pos]
                # If this vertex is not yet assigned or if the current multiplicity is lower, update.
                if assoc_array[vertex_index] == -1 or candidate_mult < best_mult[vertex_index]:
                    assoc_array[vertex_index] = sub_idx
                    best_mult[vertex_index] = candidate_mult
                    
        assoc_groups = {}
        for vertex_index, sub_idx in enumerate(assoc_array):
            # Skip any vertex that was never assigned (i.e. remains -1)
            if sub_idx == -1:
                continue
            # If this sub-array number is not yet a key, create a new list.
            if sub_idx not in assoc_groups:
                assoc_groups[sub_idx] = []
            # Append the current vertex index to the list for that key.
            assoc_groups[sub_idx].append(vertex_index)
            
        assoc_array_final = reconstruct_array(assoc_groups)






        # Create the Data object
        x = torch.from_numpy(flat_lc_feats).float()

        return Data(
            x=x, assoc = assoc_array_final
        )

In [3]:
ipath = "/vols/cms/mm1221/Data/100k/5e/train/"
vpath = "/vols/cms/mm1221/Data/100k/5e/val/"
data_train = CCV1(ipath, max_events=500, inp = 'train')
data_val = CCV1(vpath, max_events=500, inp='val')

### Loading data


  0%|                                                                                                                                                                                 | 0/3 [00:36<?, ?it/s]


Reached 500!
### Loading data


  0%|                                                                                                                                                                                 | 0/1 [00:36<?, ?it/s]

Reached 500!


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CustomStaticEdgeConv(nn.Module):
    def __init__(self, nn_module):
        super(CustomStaticEdgeConv, self).__init__()
        self.nn_module = nn_module

    def forward(self, x, edge_index):
        """
        Args:
            x (torch.Tensor): Node features of shape (N, F).
            edge_index (torch.Tensor): Predefined edges [2, E], where E is the number of edges.

        Returns:
            torch.Tensor: Node features after static edge aggregation.
        """
        row, col = edge_index  # Extract row (source) and col (target) nodes
        x_center = x[row]
        x_neighbor = x[col]

        # Compute edge features (relative)
        edge_features = torch.cat([x_center, x_neighbor - x_center], dim=-1)
        edge_features = self.nn_module(edge_features)

        # Aggregate features back to nodes
        num_nodes = x.size(0)
        node_features = torch.zeros(num_nodes, edge_features.size(-1), device=x.device)
        node_features.index_add_(0, row, edge_features)

        # Normalization (Divide by node degrees)
        counts = torch.bincount(row, minlength=num_nodes).clamp(min=1).view(-1, 1)
        node_features = node_features / counts

        return node_features
    

class Net(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=4, dropout=0.3, contrastive_dim=8):
        """
        Initializes the neural network with alternating StaticEdgeConv and GAT layers.

        Args:
            hidden_dim (int): Dimension of hidden layers.
            num_layers (int): Total number of convolutional layers (both StaticEdgeConv and GAT).
            dropout (float): Dropout rate.
            contrastive_dim (int): Dimension of the contrastive output.
            heads (int): Number of attention heads in GAT layers.
        """
        super(Net, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.contrastive_dim = contrastive_dim

        # Input encoder
        self.lc_encode = nn.Sequential(
            nn.Linear(8, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU()
        )

        # Define the network's convolutional layers, alternating between StaticEdgeConv and GAT
        self.convs = nn.ModuleList()
        for layer_idx in range(num_layers):
            # Even-indexed layers: StaticEdgeConv
            conv = CustomStaticEdgeConv(
                nn.Sequential(
                    nn.Linear(2 * hidden_dim, hidden_dim),
                    nn.ELU(),
                    nn.BatchNorm1d(hidden_dim),
                    nn.Dropout(p=dropout)
                )
            )
            self.convs.append(conv)


        # Output layer
        self.output = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ELU(),
            nn.Dropout(p=dropout),
            nn.Linear(32, 16),
            nn.ELU(),
            nn.Dropout(p=dropout),
            nn.Linear(16, contrastive_dim)
        )

    def forward(self, x, edge_index, batch):
        """
        Forward pass of the network.

        Args:
            x (torch.Tensor): Input node features of shape (N, 15).
            edge_index (torch.Tensor): Edge indices of shape (2, E).
            batch (torch.Tensor): Batch vector.

        Returns:
            torch.Tensor: Output features after processing.
            torch.Tensor: Batch vector.
        """
        # Input encoding
        x_lc_enc = self.lc_encode(x)  # Shape: (N, hidden_dim)

        # Apply convolutional layers with residual connections
        feats = x_lc_enc
        for idx, conv in enumerate(self.convs):
            feats = conv(feats, edge_index) + feats  # Residual connection

        # Final output
        out = self.output(feats)
        return out, batch


In [5]:
# Initialize model with passed hyperparameters
model = Net(
    hidden_dim=128,
    num_layers=4,
    dropout=0.3,
    contrastive_dim=16
)

k_value = 16
BS = 1

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

# Define Loss

In [13]:
import torch
import torch.nn.functional as F

def contrastive_loss_curriculum_both(embeddings, group_ids, temperature=0.1, alpha=1.0):
    """
    Computes an NT-Xent style loss that blends both positive and negative mining.
    
    For each anchor i:
      - Random positive similarity: randomly sample one index j (j != i) such that group_ids[j]==group_ids[i].
      - Hard negative similarity: max { sim(embeddings[i], embeddings[j]) : group_ids[j] != group_ids[i] }
      - Random negative similarity: similarity from a randomly chosen negative (group_ids differ)
      - Blended negative similarity: blended_neg = (1 - alpha) * rand_neg_sim + alpha * hard_neg_sim
      
    The loss per anchor is then:
         loss_i = - log( exp(random_pos_sim/temperature) / [ exp(random_pos_sim/temperature) + exp(blended_neg/temperature) ] )
    
    Anchors that lack any valid positives or negatives contribute 0.
    
    Args:
        embeddings: Tensor of shape (N, D) (raw outputs; they will be normalized inside).
        group_ids: 1D Tensor (length N) of group identifiers.
        temperature: Temperature scaling factor.
        alpha: Blending parameter for negatives (0: use only random, 1: use only hard).
        
    Returns:
        Scalar loss (mean over anchors).
    """
    print(group_ids)
    # Normalize embeddings so cosine similarity is dot product.
    norm_emb = F.normalize(embeddings, p=2, dim=1)  # shape (N, D)
    sim_matrix = norm_emb @ norm_emb.t()             # shape (N, N)
    print(sim_matrix)
    N = embeddings.size(0)
    idx = torch.arange(N, device=embeddings.device)
    print(idx)
    
    # --- Positives ---
    # Build positive mask: same group and not self.
    pos_mask = (group_ids.unsqueeze(1) == group_ids.unsqueeze(0))
    pos_mask = pos_mask & ~torch.eye(N, dtype=torch.bool, device=embeddings.device)
    print(pos_mask[0])
    valid_pos_counts = pos_mask.sum(dim=1)
    no_valid_pos = (valid_pos_counts == 0)
    
    # Randomly sample one positive index for each anchor.
    # Create random scores over the entire similarity matrix.
    rand_pos_vals = torch.rand(sim_matrix.shape, device=embeddings.device)
    # Zero out (and effectively penalize) invalid positions by subtracting a large number.
    # (Here we multiply by the mask and subtract (1 - mask) so that invalid entries are very negative.)
    rand_pos_vals = rand_pos_vals * pos_mask.float() - (1 - pos_mask.float())
    print(rand_pos_vals[0])
    rand_pos_indices = torch.argmax(rand_pos_vals, dim=1)
    rand_pos_sim = sim_matrix[idx, rand_pos_indices]
    # For anchors without any valid positive, set similarity to 0.
    rand_pos_sim = torch.where(no_valid_pos, torch.tensor(0.0, device=embeddings.device), rand_pos_sim)
    print(rand_pos_sim[0])
    
    # Use this randomly sampled similarity as the positive term.
    blended_pos = rand_pos_sim

    # --- Negatives ---
    # Build negative mask: indices with different group_ids.
    neg_mask = (group_ids.unsqueeze(1) != group_ids.unsqueeze(0))
    print(neg_mask[0])
    valid_neg_counts = neg_mask.sum(dim=1)
    no_valid_neg = (valid_neg_counts == 0)
    
    # Random negative: randomly sample one negative index per anchor.
    rand_vals = torch.rand(sim_matrix.shape, device=embeddings.device)
    rand_vals = rand_vals * neg_mask.float() - (1 - neg_mask.float())
    rand_neg_indices = torch.argmax(rand_vals, dim=1)
    rand_neg_sim = sim_matrix[idx, rand_neg_indices]
    
    # Hard negative: for each anchor, choose the negative with maximum similarity.
    sim_matrix_neg = sim_matrix.masked_fill(~neg_mask, -float('inf'))
    hard_neg_sim, _ = sim_matrix_neg.max(dim=1)
    hard_neg_sim = torch.where(no_valid_neg, torch.tensor(-1.0, device=embeddings.device), hard_neg_sim)
    
    # Blend the negatives.
    blended_neg = (1 - alpha) * rand_neg_sim + alpha * hard_neg_sim

    # Compute loss per anchor.
    loss = -torch.log(
        torch.exp(blended_pos / temperature) / 
        (torch.exp(blended_pos / temperature) + torch.exp(blended_neg / temperature))
    )
    # For anchors with no valid negatives, set loss to 0.
    loss = loss.masked_fill(no_valid_neg, 0.0)
    
    return loss.mean()


def contrastive_loss_curriculum(embeddings, group_ids, temperature=0.1, alpha=1.0):
    """
    Curriculum loss that uses both positive and negative blending.
    
    Delegates to contrastive_loss_curriculum_both.
    
    Args:
        embeddings: Tensor of shape (N, D).
        pos_indices: 1D Tensor (length N).
        group_ids: 1D Tensor (length N).
        temperature: Temperature scaling factor.
        alpha: Blending parameter.
        
    Returns:
        Scalar loss.
    """
    return contrastive_loss_curriculum_both(embeddings, group_ids, temperature, alpha)

In [14]:
from tqdm import tqdm
def train_new(train_loader, model, optimizer, device, k_value, alpha):
    model.train()
    total_loss = torch.zeros(1, device=device)
    for data in tqdm(train_loader, desc="Training"):
        data = data.to(device)
        optimizer.zero_grad()
        
        # Convert data.assoc to tensor if needed.
        if isinstance(data.assoc, list):
            if isinstance(data.assoc[0], list):
                assoc_tensor = torch.cat([torch.tensor(a, dtype=torch.int64, device=data.x.device)
                                          for a in data.assoc])
            else:
                assoc_tensor = torch.tensor(data.assoc, device=data.x.device)
        else:
            assoc_tensor = data.assoc

        edge_index = knn_graph(data.x[:, :3], k=k_value, batch=data.x_batch)
        embeddings, _ = model(data.x, edge_index, data.x_batch)
        
        # Partition batch by event.
        batch_np = data.x_batch.detach().cpu().numpy()
        _, counts = np.unique(batch_np, return_counts=True)
        
        loss_event_total = torch.zeros(1, device=device)
        start_idx = 0
        for count in counts:
            end_idx = start_idx + count
            event_embeddings = embeddings[start_idx:end_idx]
            event_group_ids = assoc_tensor[start_idx:end_idx]
            loss_event = contrastive_loss_curriculum(event_embeddings,
                                                     event_group_ids, temperature=0.1, alpha=alpha)
            loss_event_total += loss_event
            start_idx = end_idx
        
        loss = loss_event_total / len(counts)
        loss.backward()
        total_loss += loss
        optimizer.step()
    return total_loss / len(train_loader.dataset)

@torch.no_grad()
def test_new(test_loader, model, device, k_value):
    model.eval()
    total_loss = torch.zeros(1, device=device)
    for data in tqdm(test_loader, desc="Validation"):
        data = data.to(device)
        
        if isinstance(data.assoc, list):
            if isinstance(data.assoc[0], list):
                assoc_tensor = torch.cat([torch.tensor(a, dtype=torch.int64, device=data.x.device)
                                          for a in data.assoc])
            else:
                assoc_tensor = torch.tensor(data.assoc, device=data.x.device)
        else:
            assoc_tensor = data.assoc
        
        edge_index = knn_graph(data.x[:, :3], k=k_value, batch=data.x_batch)
        embeddings, _ = model(data.x, edge_index, data.x_batch)
        
        batch_np = data.x_batch.detach().cpu().numpy()
        _, counts = np.unique(batch_np, return_counts=True)
        
        loss_event_total = torch.zeros(1, device=device)
        start_idx = 0
        for count in counts:
            end_idx = start_idx + count
            event_embeddings = embeddings[start_idx:end_idx]
            event_group_ids = assoc_tensor[start_idx:end_idx]
            loss_event = contrastive_loss_curriculum(event_embeddings,
                                                     event_group_ids, temperature=0.1, alpha=1)
            loss_event_total += loss_event
            start_idx = end_idx
        total_loss += loss_event_total / len(counts)
    return total_loss / len(test_loader.dataset)


In [15]:

device = torch.device('cpu')
# Load DataLoader with current batch_size
train_loader = DataLoader(data_train, batch_size=BS, shuffle=False, follow_batch=['x'])
val_loader = DataLoader(data_val, batch_size=BS, shuffle=False, follow_batch=['x'])

# Train and evaluate the model for the specified number of epochs
best_val_loss = float('inf')

# Store train and validation losses for all epochs
train_losses = []
val_losses = []

output_dir = '/vols/cms/mm1221/hgcal/elec5New/LC/NegativeMining/results/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
epochs = 20
for epoch in range(epochs):
    # For epochs 1 to 150, gradually increase alpha from 0 to 1.
    # From epoch 151 onward, set alpha = 1 (fully hard negatives).
    if epoch < 10:
        alpha = (epoch + 1) / 10.0
    else:
        alpha = 1.0

    print(f"Epoch {epoch+1}/{epochs} | Alpha: {alpha:.2f}")
    train_loss = train_new(train_loader, model, optimizer, device, k_value, alpha)
    val_loss = test_new(val_loader, model, device, k_value)

    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    scheduler.step()

    # Save best model if validation loss improves.
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        no_improvement_epochs = 0
        torch.save(model.state_dict(), os.path.join(output_dir, 'best_model.pt'))
    else:
        no_improvement_epochs += 1

    # Save intermediate checkpoint.
    state_dicts = {'model': model.state_dict(),
                   'opt': optimizer.state_dict(),
                   'lr': scheduler.state_dict()}
    torch.save(state_dicts, os.path.join(output_dir, f'epoch-{epoch+1}.pt'))

    print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss.item():.8f}, Validation Loss: {val_loss.item():.8f}")
    if no_improvement_epochs >= patience:
        print(f"Early stopping triggered. No improvement for {patience} epochs.")
        break

# Save training history.
import pandas as pd
results_df = pd.DataFrame({
    'epoch': list(range(1, len(train_losses) + 1)),
    'train_loss': train_losses,
    'val_loss': val_losses
})
results_df.to_csv(os.path.join(output_dir, 'continued_training_loss.csv'), index=False)
print(f"Saved loss curves to {os.path.join(output_dir, 'continued_training_loss.csv')}")

# Save final model.
torch.save(model.state_dict(), os.path.join(output_dir, 'final_model.pt'))
print("Training complete. Final model saved.")

Epoch 1/20 | Alpha: 0.10


Training:   0%|▏                                                                                                                                                            | 1/818 [00:00<01:27,  9.31it/s]

tensor([0, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0,
        1, 0, 1, 2, 2, 0, 0, 0, 2, 0, 1, 0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 1,
        2, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 2, 1, 2, 2, 2,
        2, 2, 2, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 0, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 1, 0, 2,
        1, 0, 2, 2, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0,
        2, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 1, 0, 0, 2, 1, 2, 2, 2, 2, 0, 1, 0, 0,
        2, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 2,
        2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 2,
        0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2,
        2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1, 1, 2, 0, 1, 1, 0])
tensor([[1.0000, 0.9054, 0.9409,  ..., 0.9268, 0.9157, 0.9070],
        [0.9054, 1.0000, 0.8352,  ..., 0.9560, 0.9517, 0.9140],
      

tensor([2, 1, 1, 1, 2, 0, 0, 0, 1, 1, 1, 2, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 2, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 1, 2, 2, 0, 0, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 1, 2, 2, 0, 2, 0, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2,
        0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 2, 2,
        0, 2, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 0, 0, 2,
        2, 1, 2, 2, 0, 0, 0, 0, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 0, 0,
        1, 2, 2, 2, 2, 2, 0, 2, 0, 1, 0, 0, 0, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 2, 0, 2, 0, 0, 0, 0,

Training:   0%|▌                                                                                                                                                            | 3/818 [00:00<01:59,  6.82it/s]

...

Training:   1%|██                                                                                                                                                          | 11/818 [00:01<02:32,  5.30it/s]

tensor([0, 2, 3, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 0, 3, 1, 2, 2, 2, 0, 3, 0, 2,
        2, 3, 3, 3, 0, 1, 3, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 0, 3, 1, 1, 3, 1, 2,
        2, 2, 2, 0, 1, 1, 0, 0, 2, 2, 0, 0, 3, 0, 3, 1, 0, 1, 1, 0, 2, 2, 0, 2,
        2, 3, 2, 1, 2, 0, 3, 2, 0, 3, 3, 1, 1, 1, 1, 3, 3, 0, 2, 2, 2, 0, 1, 3,
        0, 1, 3, 2, 2, 2, 0, 3, 1, 1, 3, 3, 1, 0, 1, 1, 3, 1, 1, 1, 0, 2, 0, 2,
        0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 0, 3, 3, 1, 1, 1, 1, 3, 2,
        2, 2, 2, 2, 2, 0, 1, 1, 1, 2, 0, 3, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2,
        0, 0, 0, 1, 3, 3, 1, 1, 0, 3, 1, 1, 3, 2, 2, 0, 1, 2, 0, 2, 0, 2, 0, 0,
        2, 0, 0, 2, 2, 0, 3, 2, 2, 2, 0, 0, 2, 2, 2, 2, 3, 0, 3, 3, 3, 1, 1, 1,
        1, 1, 3, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 1, 2, 2, 1, 1, 0, 2, 0, 2,
        2, 2, 2, 2, 2, 2, 1, 1, 3, 3, 0, 0, 1, 3, 0, 1, 1, 1, 1, 0, 2, 2, 0, 0,
        0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 3, 0, 3, 3, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 2,

Training:   1%|██▎                                                                                                                                                         | 12/818 [00:02<02:24,  5.59it/s]

tensor([2, 2, 2, 1, 0, 2, 3, 0, 2, 2, 2, 1, 3, 2, 3, 0, 3, 2, 2, 2, 1, 1, 1, 0,
        0, 3, 3, 3, 1, 0, 1, 0, 2, 2, 2, 1, 1, 1, 1, 3, 3, 0, 3, 1, 1, 2, 2, 1,
        1, 0, 2, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 3, 0, 3, 3, 3, 2, 2, 2, 2, 2, 2,
        0, 3, 3, 3, 2, 2, 2, 1, 1, 2, 1, 3, 3, 2, 3, 0, 0, 3, 1, 0, 3, 2, 2, 0,
        2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 0, 0, 1, 0, 0, 3, 0, 0, 3, 3, 0,
        0, 3, 3, 2, 2, 2, 0, 2, 2, 0, 1, 2, 2, 2, 1, 1, 1, 2, 1, 0, 0, 3, 0, 0,
        3, 0, 0, 0, 0, 3, 1, 1, 2, 0, 2, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 0, 0,
        3, 0, 0, 3, 0, 1, 0, 3, 0, 3, 3, 0, 2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 1, 1,
        2, 1, 0, 0, 3, 0, 0, 3, 0, 1, 0, 0, 3, 1, 0, 3, 3, 3, 3, 3, 2, 2, 2, 2,
        2, 2, 2, 2, 0, 2, 2, 3, 1, 2, 2, 2, 1, 1, 2, 0, 0, 0, 3, 0, 3, 0, 0, 3,
        3, 0, 0, 3, 1, 3, 3, 2, 2, 2, 2, 2, 3, 3, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1,
        3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 0, 0, 1, 0, 3, 3, 1, 2, 2, 2, 2, 1, 1, 2,
        2, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 

In [9]:
print(data_train[0].assoc)

[0, 1, 2, 0, 0, 1, 2, 0, 1, 0, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0, 1, 0, 1, 2, 2, 0, 0, 0, 2, 0, 1, 0, 2, 2, 2, 0, 0, 0, 0, 1, 1, 0, 0, 1, 2, 2, 2, 0, 1, 1, 0, 2, 2, 2, 2, 0, 1, 2, 1, 0, 0, 0, 0, 2, 1, 2, 2, 2, 2, 2, 2, 0, 0, 1, 1, 0, 0, 2, 2, 2, 2, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 1, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 1, 0, 1, 0, 0, 0, 2, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 1, 0, 0, 2, 1, 2, 2, 2, 2, 0, 1, 0, 0, 2, 1, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 2, 0, 1, 1, 0, 2, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1, 1, 2, 0, 1, 1, 0]
